I obtained a 2nd dataset which looks better than the first one. I will develop a second model and compare it to the first one to choose the better model.

### Loading primary libraries

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt              
import seaborn as sns   

## Loading the new dataset

In [21]:
df = pd.read_csv('../data/Vehicle Price.csv')
print(f'CSV file loaded succesfully!')
df.head()

CSV file loaded succesfully!


,name,description,make,model,year,price,engine,cylinders,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,2024,74600.0,24V GDI DOHC Twin Turbo,6.0,Gasoline,10.0,8-Speed Automatic,Series II,SUV,4.0,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,2024,50170.0,OHV,6.0,Gasoline,1.0,8-Speed Automatic,Laredo,SUV,4.0,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,2024,96410.0,"6.2L V-8 gasoline direct injection, variable v...",8.0,Gasoline,0.0,Automatic,Denali,SUV,4.0,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,2023,46835.0,16V MPFI OHV,8.0,Gasoline,32.0,8-Speed Automatic,Pursuit,SUV,4.0,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,2024,81663.0,24V DDI OHV Turbo Diesel,6.0,Diesel,10.0,6-Speed Automatic,Laramie,Pickup Truck,4.0,Silver,Black,Four-wheel Drive


## EDA & Preprocess

In [22]:
print(f'The dataframe is composed by {df.shape[0]} observations & {df.shape[1]} columns.')

The dataframe is composed by 1002 observations & 17 columns.


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1002 non-null   object 
 1   description     946 non-null    object 
 2   make            1002 non-null   object 
 3   model           1002 non-null   object 
 4   year            1002 non-null   int64  
 5   price           979 non-null    float64
 6   engine          1000 non-null   object 
 7   cylinders       897 non-null    float64
 8   fuel            995 non-null    object 
 9   mileage         968 non-null    float64
 10  transmission    1000 non-null   object 
 11  trim            1001 non-null   object 
 12  body            999 non-null    object 
 13  doors           995 non-null    float64
 14  exterior_color  997 non-null    object 
 15  interior_color  964 non-null    object 
 16  drivetrain      1002 non-null   object 
dtypes: float64(4), int64(1), object(1

In [24]:
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 283
Every null value:
name                0
description        56
make                0
model               0
year                0
price              23
engine              2
cylinders         105
fuel                7
mileage            34
transmission        2
trim                1
body                3
doors               7
exterior_color      5
interior_color     38
drivetrain          0
dtype: int64


There are some null values & some columns which are in wrong types. Preprocess will begin with correcting null values.

First, i noticed that there are some null values on the Price column. This is a essential column, so if there are no price, the observation should not exist. I will drop the observations whose price column is empty.

In [26]:
# Droping null values from the Price column.
df = df.dropna(subset=['price'])
print(f'Null values from the Price column dropped successfully!')

Null values from the Price column dropped successfully!


In [27]:
# Checking new number of null values.
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 256
Every null value:
name                0
description        56
make                0
model               0
year                0
price               0
engine              2
cylinders         102
fuel                7
mileage            34
transmission        2
trim                1
body                3
doors               7
exterior_color      5
interior_color     37
drivetrain          0
dtype: int64


Cylinders is an important column and it has a great amount of null values. I will do some research of the column to take an action.

In [28]:
df['cylinders'].unique()

array([ 6.,  8.,  4., nan,  3.,  0.])

As we know, it is impossible for a car to have 0 cylinders, so thats a null value too.

In [31]:
print('Value Counts of the Cylinders column:')
print(df['cylinders'].value_counts())

Value Counts of the Cylinders column:
cylinders
4.0    489
6.0    272
8.0     87
3.0     27
0.0      2
Name: count, dtype: int64


The mayority of cars have 4 cylinders, and other amount has 6. First, I will start by turning the 0 values into NaN.

In [33]:
df['cylinders'] = df['cylinders'].replace(0.0, np.nan)

100 null values are too many null values too drop or impute in a single columns. For this reason, i will do a random ponderated impute.

In [36]:
# Distribution in percentage of values.
distribution_cyl = df['cylinders'].value_counts(normalize=True)
print('Distribution of Cylinders (%):')
print(distribution_cyl)

Distribution of Cylinders (%):
cylinders
4.0    0.558857
6.0    0.310857
8.0    0.099429
3.0    0.030857
Name: proportion, dtype: float64


In [38]:
# Transforming the distribution to list.
values = distribution_cyl.index.tolist()
weights = distribution_cyl.values.tolist()

# Obtaining indexs of NaN values.
nan_index = df[df['cylinders'].isna()].index

# Generating random values based on the distribution.
random_values = np.random.choice(values, size=len(nan_index), p=weights)

# Imputing the random values into the NaN indexs.
df.loc[nan_index, 'cylinders'] = random_values
print(f'Null values from the Cylinders column imputed successfully!')


Null values from the Cylinders column imputed successfully!


In [39]:
print('Value Counts of the Cylinders column:')
print(df['cylinders'].value_counts())

Value Counts of the Cylinders column:
cylinders
4.0    556
6.0    300
8.0     96
3.0     27
Name: count, dtype: int64


In [40]:
# Checking new number of null values.
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 154
Every null value:
name               0
description       56
make               0
model              0
year               0
price              0
engine             2
cylinders          0
fuel               7
mileage           34
transmission       2
trim               1
body               3
doors              7
exterior_color     5
interior_color    37
drivetrain         0
dtype: int64


Next stop in handling null values is the description column. Description is a text free columns (object) that will not be easy for the ML library to use. It has some null values and the columns doesn't have much importance by itself. Therefore, the column will be dropped.

In [41]:
# Droping the description column.
df = df.drop(columns=['description'])
print(f'Description column dropped successfully!')

Description column dropped successfully!


In [42]:
# Checking new number of null values.
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 98
Every null value:
name               0
make               0
model              0
year               0
price              0
engine             2
cylinders          0
fuel               7
mileage           34
transmission       2
trim               1
body               3
doors              7
exterior_color     5
interior_color    37
drivetrain         0
dtype: int64


Checking values distribution for the next columns with null values.

In [46]:
print(f'Fuel unique values: {df['fuel'].unique()}')
print(f'Fuel distribution:\n{df['fuel'].value_counts()}')

Fuel unique values: ['Gasoline' 'Diesel' 'Hybrid' 'Electric' 'E85 Flex Fuel'
 'PHEV Hybrid Fuel' nan 'Diesel (B20 capable)']
Fuel distribution:
fuel
Gasoline                647
Hybrid                  135
Electric                 96
Diesel                   72
PHEV Hybrid Fuel         16
E85 Flex Fuel             5
Diesel (B20 capable)      1
Name: count, dtype: int64


Fuel columns has only 7 null values that will be imputed by mode.

In [47]:
# Imputing null values in the fuel column by mode.
df['fuel'] = df['fuel'].fillna(df['fuel'].mode()[0])
print(f'Null values from the Fuel column imputed successfully!')

Null values from the Fuel column imputed successfully!


Repeating process with engine column.

In [52]:
print(f'Number of unique values in the engine column: {df['engine'].nunique()}')
# Showing the distribution of the engine column.
print(f'Engine unique values: {df['engine'].value_counts()}')


Number of unique values in the engine column: 100
Engine unique values: engine
16V GDI DOHC Turbo                                           120
16V GDI DOHC Turbo Hybrid                                     92
c                                                             92
24V MPFI DOHC                                                 53
16V MPFI OHV                                                  46
                                                            ... 
16V MPFI DOHC Hybrid                                           1
o                                                              1
4L V-8 VVT variable valve control, regular unleaded, engi      1
DOHC, D-CVVT variable valve control, regular unleaded, en      1
8 gasoline direct injection, variable valve control, regu      1
Name: count, Length: 100, dtype: int64


In [53]:
# Imputing null values in the fuel column by mode.
df['engine'] = df['engine'].fillna(df['engine'].mode()[0])
print(f'Null values from the Engine column imputed successfully!')

Null values from the Engine column imputed successfully!


In [54]:
# Checking new number of null values.
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 89
Every null value:
name               0
make               0
model              0
year               0
price              0
engine             0
cylinders          0
fuel               0
mileage           34
transmission       2
trim               1
body               3
doors              7
exterior_color     5
interior_color    37
drivetrain         0
dtype: int64


Checking data from mileage column.

In [56]:
# Number of unique values.
print(f'Number of unique values in the mileage column: {df['mileage'].nunique()}')
# Distribution.
print(f'Mileage unique values: {df['mileage'].value_counts()}')

Number of unique values in the mileage column: 91
Mileage unique values: mileage
5.0      113
0.0      109
10.0     103
1.0       58
6.0       50
        ... 
241.0      1
66.0       1
41.0       1
141.0      1
296.0      1
Name: count, Length: 91, dtype: int64


In [57]:
# Showing values whre mileage is null.
print(f'Null values in the Mileage column:\n{df[df['mileage'].isna()]}')

Null values in the Mileage column:
                                           name        make  \
27             2024 Chrysler Pacifica Touring-L    Chrysler   
47               2024 Subaru Outback Wilderness      Subaru   
63           2024 Jeep Grand Cherokee L Limited        Jeep   
73                      2024 Jeep Wagoneer Base        Jeep   
84           2024 Jeep Grand Cherokee L Limited        Jeep   
87                    2024 Buick Envista Avenir       Buick   
93                          2024 Ford Ranger XL        Ford   
177               2024 Lincoln Nautilus Reserve     Lincoln   
217                2024 Jeep Wrangler 4xe Sport        Jeep   
219                 2024 Honda Prologue TOURING       Honda   
225                 2024 Volkswagen Taos 1.5T S  Volkswagen   
234                     2024 Jeep Wagoneer Base        Jeep   
261                2024 Jeep Wagoneer Series II        Jeep   
262                 2023 Jeep Compass Trailhawk        Jeep   
365                 

In [62]:
df.iloc[93]

name                   2024 Ford Ranger XL
make                                  Ford
model                               Ranger
year                                  2024
price                              38740.0
engine                  16V GDI DOHC Turbo
cylinders                              4.0
fuel                              Gasoline
mileage                                NaN
transmission            10-Speed Automatic
trim                                    XL
body                          Pickup Truck
doors                                  4.0
exterior_color    Carbonized Gray Metallic
interior_color                       Ebony
drivetrain                Four-wheel Drive
Name: 93, dtype: object

After checking for some observations, the mayority of null values on this column are new cars (based on their price). Therefore, i will change Nan for 0.

In [63]:
# Turning NaN values into 0.
df['mileage'] = df['mileage'].fillna(0)
print(f'Null values from the Mileage column imputed successfully!')

Null values from the Mileage column imputed successfully!


Continuing with interior color.

In [66]:
# Unique values of interior color.
print(f'Number of unique values on Interior Color: {df['interior_color'].nunique()}')
# Distribution of interior color.
print(f'Interior Color unique values: {df['interior_color'].value_counts()}')

Number of unique values on Interior Color: 90
Interior Color unique values: interior_color
Black           497
Global Black     83
Gray             76
Jet Black        45
Ebony            38
               ... 
Caramel           1
gray              1
Dark Palazzo      1
Gray/Black        1
Navy Pier         1
Name: count, Length: 90, dtype: int64


Most of the car has black as interior color. I don't think this is one the most important columns, so i will impute the null data with mode.

In [67]:
# Imputing null values in the interior color column by mode.
df['interior_color'] = df['interior_color'].fillna(df['interior_color'].mode()[0])
print(f'Null values from the Interior Color column imputed successfully!')

Null values from the Interior Color column imputed successfully!


In [68]:
# Checking new number of null values.
print(f'Number of null values: {df.isna().sum().sum()}')
print(f'Every null value:\n{df.isna().sum()}')

Number of null values: 18
Every null value:
name              0
make              0
model             0
year              0
price             0
engine            0
cylinders         0
fuel              0
mileage           0
transmission      2
trim              1
body              3
doors             7
exterior_color    5
interior_color    0
drivetrain        0
dtype: int64


Finally, all this columns have less than 10 null values, so all of them will be imputed by mode.

In [69]:
# Imputing null values in the transmission, trim, body, doors and exterior color columns by mode.
columns_to_impute = ['transmission', 'trim', 'body', 'doors', 'exterior_color']
for column in columns_to_impute:
    df[column] = df[column].fillna(df[column].mode()[0])
    print(f'Null values from the {column} column imputed successfully!')

Null values from the transmission column imputed successfully!
Null values from the trim column imputed successfully!
Null values from the body column imputed successfully!
Null values from the doors column imputed successfully!
Null values from the exterior_color column imputed successfully!


In [70]:
print(f'Number of null values: {df.isna().sum().sum()}')

Number of null values: 0


I will finish the preprocess section changing some float types columns to int columns.

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 979 entries, 0 to 1001
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            979 non-null    object 
 1   make            979 non-null    object 
 2   model           979 non-null    object 
 3   year            979 non-null    int64  
 4   price           979 non-null    float64
 5   engine          979 non-null    object 
 6   cylinders       979 non-null    float64
 7   fuel            979 non-null    object 
 8   mileage         979 non-null    float64
 9   transmission    979 non-null    object 
 10  trim            979 non-null    object 
 11  body            979 non-null    object 
 12  doors           979 non-null    float64
 13  exterior_color  979 non-null    object 
 14  interior_color  979 non-null    object 
 15  drivetrain      979 non-null    object 
dtypes: float64(4), int64(1), object(11)
memory usage: 162.3+ KB


In [72]:
# Selecting float columns to convert to int.
float_columns = ['cylinders','mileage','doors']
# Converting float columns to int.
for col in float_columns:
    df[col] = df[col].astype(int)
    print(f'Column {col} converted to int successfully!')


Column cylinders converted to int successfully!
Column mileage converted to int successfully!
Column doors converted to int successfully!
